# Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import os
import warnings
import pickle
warnings.filterwarnings("ignore")
%matplotlib inline

# Bootstrap

In [9]:
import sys
import os
from dotenv import load_dotenv

# Find and load the .env file from the current or parent directories
load_dotenv()

# Access the environment variable using os.getenv()
PROJECT_PATH = os.getenv('PROJECT_PATH')
CONFIG_FILE = f"{PROJECT_PATH}/src/config.json"

In [10]:
from google.colab import drive
import json

drive.mount('/content/drive')

with open(CONFIG_FILE, 'r') as f:
    project_config = json.load(f)
    project_config.pop('_comment', None)
    project_config.pop('_note', None)
    f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
np.random.seed(31071967)

# Senity check

In [12]:
for item in project_config:
    print(f"{item} = {project_config[item]}")

data_directory = data/
notebooks_directory = notebooks/
src_directory = src/
pickles_directory = pickles/
output_directory = output/
notebook1 = 1stocks_dataprep_xgb.ipynb
notebook2 = 2stocks_lstm
notebook3 = 3stocks_gru.ipynb
notebook4 = 4stocks_cascade_lstm_gru.ipynb
notebook5 = 5stocks_parallel_lstm_gru.ipynb
notebook6 = 6stocks_prediction.ipynb
output1 = 
output2 = 
output3 = 
output4 = 
output5 = 
pickle1 = 
pickle2 = 
pickle3 = 
pickle4 = 
pickle_final_df = 
pickle_final_model = 
chain_notebooks = 0
split_df = 1
outliers_alg = 
imputer_alg = 
model_alg = 


# Start coding here